In [21]:
import os
import csv
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
from matplotlib import pyplot as plt
from PIL import Image
import cv2
#import matplotlib.patches as patches

In [22]:
def Dataset_loader(DIR, RESIZE, sigmaX=10):
    IMG = []
    read = lambda imname: np.asarray(Image.open(imname).convert("RGB"))
    for IMAGE_NAME in tqdm(os.listdir(DIR)):
        PATH = os.path.join(DIR,IMAGE_NAME)
        _, ftype = os.path.splitext(PATH)
        if ftype == ".png":
            img = read(PATH)
           
            img = cv2.resize(img, (RESIZE,RESIZE))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
           
            IMG.append(np.array(img))
    return IMG

In [23]:
x_covid19_train = np.array(Dataset_loader(r"C:\Users\sbagh\Downloads\fcomp\Lung Segmentation Data\Lung Segmentation Data\Train\COVID-19\lung masks",224))
x_noncovid_train = np.array(Dataset_loader(r"C:\Users\sbagh\Downloads\fcomp\Lung Segmentation Data\Lung Segmentation Data\Train\Non-COVID\lung masks",224))
x_normal_train = np.array(Dataset_loader(r"C:\Users\sbagh\Downloads\fcomp\Lung Segmentation Data\Lung Segmentation Data\Train\Normal\lung masks",224))


100%|██████████| 6849/6849 [00:11<00:00, 609.18it/s]


In [ ]:
x_covid19_train.shape
#plt.imshow(x_covid19_train[5])

In [24]:
x_covid19_val = np.array(Dataset_loader(r"C:\Users\sbagh\Downloads\fcomp\Lung Segmentation Data\Lung Segmentation Data\Val\COVID-19\lung masks",224))
x_noncovid_val = np.array(Dataset_loader(r"C:\Users\sbagh\Downloads\fcomp\Lung Segmentation Data\Lung Segmentation Data\Val\Non-COVID\lung masks",224))
x_normal_val = np.array(Dataset_loader(r"C:\Users\sbagh\Downloads\fcomp\Lung Segmentation Data\Lung Segmentation Data\Val\Normal\lung masks",224))


100%|██████████| 1712/1712 [00:02<00:00, 588.47it/s]


In [25]:
#Xtest
x_covid19_test = np.array(Dataset_loader(r"C:\Users\sbagh\Downloads\fcomp\Lung Segmentation Data\Lung Segmentation Data\Test\COVID-19\lung masks",224))
x_noncovid_test = np.array(Dataset_loader(r"C:\Users\sbagh\Downloads\fcomp\Lung Segmentation Data\Lung Segmentation Data\Test\Non-COVID\lung masks",224))
x_normal_test = np.array(Dataset_loader(r"C:\Users\sbagh\Downloads\fcomp\Lung Segmentation Data\Lung Segmentation Data\Test\Normal\lung masks",224))


100%|██████████| 2140/2140 [00:03<00:00, 626.00it/s]


In [26]:
ytrain_covid19= np.zeros(len(x_covid19_train))
ytrain_noncovid = np.ones(len(x_noncovid_train))
ytrain_normal = (np.ones(len(x_normal_train))) * 2

yval_covid19 = np.zeros(len(x_covid19_val))
yval_noncovid = np.ones(len(x_noncovid_val))
yval_normal = np.ones(len(x_normal_val)) * 2

ytest_covid19 = np.zeros(len(x_covid19_test))
ytest_noncovid = np.ones(len(x_noncovid_test))
ytest_normal = np.ones(len(x_normal_test)) * 2



In [27]:
X_train=np.concatenate((x_covid19_train,x_noncovid_train,x_normal_train))
y_train=np.concatenate((ytrain_covid19,ytrain_noncovid,ytrain_normal))
X_val=np.concatenate((x_covid19_val,x_noncovid_val,x_normal_val))
y_val=np.concatenate((yval_covid19,yval_noncovid,yval_normal))
X_test=np.concatenate((x_covid19_test,x_noncovid_test,x_normal_test))
y_test=np.concatenate((ytest_covid19,ytest_noncovid,ytest_normal))

In [ ]:
# Data Augmentation
from keras.preprocessing.image import ImageDataGenerator
BATCH_SIZE = 16

train_generator = ImageDataGenerator(
        zoom_range=2,  # set range for random zoom
        rotation_range = 90,
        horizontal_flip=True,  # randomly flip images
        vertical_flip=True,  # randomly flip images
    )

In [ ]:
X_train.shape
X_val.shape
y_train.shape
y_val.shape

In [28]:
from keras.utils import to_categorical
y_train = to_categorical(y_train, num_classes=3)
y_val = to_categorical(y_val, num_classes=3)
y_test = to_categorical(y_test, num_classes=3)

In [29]:
#CNN

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,BatchNormalization,Dropout
from keras.utils import to_categorical

# Define the model
model = Sequential()

# Add convolutional layer(s)
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(224, 224, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))

# Flatten the output from convolutional layers
model.add(Flatten())

# Add a dense layer for classification
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_10 (Conv2D)          (None, 222, 222, 16)      160       
                                                                 
 batch_normalization (BatchN  (None, 222, 222, 16)     64        
 ormalization)                                                   
                                                                 
 conv2d_11 (Conv2D)          (None, 220, 220, 32)      4640      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 110, 110, 32)     0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 110, 110, 32)      0         
                                                                 
 conv2d_12 (Conv2D)          (None, 108, 108, 64)     

In [ ]:
#CNN LSTM

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from keras.utils import to_categorical
# Define the model
model = Sequential()

# Add convolutional layer(s)
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Add a TimeDistributed layer with LSTM
model.add(TimeDistributed(Flatten()))
model.add(LSTM(64, activation='tanh', return_sequences=True))
model.add(Flatten())

# Add a dense layer for classification
model.add(Dense(64, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print the model summary
model.summary()

In [31]:
# Fit the model to the training data
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Convert the predictions from one-hot encoded vectors to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Print some sample predictions
print('Sample predictions:')
for i in range(10):
    print('Actual:', np.argmax(y_test[i]), 'Predicted:', y_pred_labels[i])


Epoch 1/10
679/679 [==============================] - 1852s 3s/step - loss: 2.6686 - accuracy: 0.3466 - val_loss: 1.0977 - val_accuracy: 0.3513
Epoch 2/10
679/679 [==============================] - 12270s 18s/step - loss: 1.0977 - accuracy: 0.3527 - val_loss: 1.0977 - val_accuracy: 0.3513
Epoch 3/10
 87/679 [==>...........................] - ETA: 47:02 - loss: 1.0971 - accuracy: 0.3570

In [ ]:
#Auto Encoder

from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Input

# Define the encoder
input_img = Input(shape=(224, 224, 1))
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)
encoded_shape = encoded.shape[1:] # Get the shape of the encoded representation

# Define the decoder
x = Conv2D(128, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

# Combine the encoder and decoder into an autoencoder
autoencoder = Model(input_img, decoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=10, batch_size=32, validation_data=(X_val, X_val))

# Extract the encoded features
encoder = Model(input_img, encoded)
X_train_encoded = encoder.predict(X_train)
X_val_encoded = encoder.predict(X_val)
X_test_encoded = encoder.predict(X_test)

# Add a dense layer for classification
model = Sequential()
model.add(Dense(64, activation='relu', input_dim=encoded_shape))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Fit the model to the training data
history = model.fit(X_train_encoded, y_train, epochs=10, batch_size=32, validation_data=(X_val_encoded, y_val))

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test_encoded, y_test)
print('Test accuracy:', test_acc)

# Make predictions on the test data
y_pred = model.predict(X_test_encoded)

# Convert the predictions from one-hot encoded vectors to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Print some sample predictions
print('Sample predictions:')
for i in range(10):
    print('Actual:', np.argmax(y_test[i]), 'Predicted:', y_pred_labels[i])


In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(X_test_encoded, y_test)
print('Test accuracy:', test_acc)

# Make predictions on the test data
y_pred = model.predict(X_test_encoded)

# Convert the predictions from one-hot encoded vectors to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Print some sample predictions
print('Sample predictions:')
for i in range(10):
    print('Actual:', np.argmax(y_test[i]), 'Predicted:', y_pred_labels[i])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Select a sample from the test set
sample_idx = 1
sample = X_test[sample_idx]
true_label = np.argmax(y_test[sample_idx])

# Make a prediction on the sample
pred = model.predict(np.expand_dims(sample, axis=0))[0]
pred_label = np.argmax(pred)

# Plot the sample and its predicted label
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.imshow(sample.squeeze(), cmap='gray')
ax1.axis('off')
ax1.set_title('Sample Image')
ax2.bar(range(3), pred)
ax2.set_xticks(range(3))
ax2.set_xticklabels(['Covid', 'Non-Covid', 'Normal'])
ax2.set_title('Predicted Class Probabilities')
ax2.set_ylim([0, 1])
ax2.set_xlabel('Class')
ax2.set_ylabel('Probability')
plt.show()

print('True label:', true_label)
print('Predicted label:', pred_label)


In [ ]:

# Plot the training, validation, and test accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

# Plot the training, validation, and test loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()


In [ ]:
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Convert test set labels to class labels
y_true_labels = np.argmax(y_test, axis=1)

# Compute confusion matrix
confusion_mtx = confusion_matrix(y_true_labels, y_pred_labels)

# Define class labels
class_names = ['Covid', 'Non Covid', 'Normal']

# Plot confusion matrix
plt.imshow(confusion_mtx, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names, rotation=45)
plt.yticks(tick_marks, class_names)
fmt = 'd'
thresh = confusion_mtx.max() / 2.
for i, j in itertools.product(range(confusion_mtx.shape[0]), range(confusion_mtx.shape[1])):
    plt.text(j, i, format(confusion_mtx[i, j], fmt),
             horizontalalignment="center",
             color="white" if confusion_mtx[i, j] > thresh else "black")
plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.show()
